In [1]:
from subprocess import Popen, PIPE
from pathlib import Path
import time
import shutil
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
JAVA_PATH = 'java'
JAR_PATH = 'target/sem11-java-bonus-1.0-SNAPSHOT-jar-with-dependencies.jar'
ARG_PREFIX = [JAVA_PATH, '-jar', JAR_PATH]
SERVER_TYPES = ['blocking', 'nonblocking', 'async']
PORT = '4444'

In [3]:
fixed_request_size = 4096
fixed_n_clients = 20
fixed_request_delay_ms = 50
fixed_n_requests_per_client = 20

In [4]:
experiment_descs = pd.DataFrame(columns=['request_size', 'n_clients', 'request_delay_ms', 'n_requests_per_client'])

In [5]:
def run_experiment(request_size: int, n_clients: int, request_delay_ms: int, n_requests_per_client: int):
    global experiment_descs
    experiment_id = len(experiment_descs)
    experiment_descs.loc[experiment_id] = {
        'request_size': request_size,
        'n_clients': n_clients,
        'request_delay_ms': request_delay_ms,
        'n_requests_per_client': n_requests_per_client,
    }

    client_procs = []
    dir = Path('results') / 'different_computers' / f'{experiment_id:04}'
    dir.mkdir(parents=True, exist_ok=True)

    with (dir / 'clients.csv').open('w') as client_out:
        client_out.write('total_ns\n')
        for i in range(n_clients):
            args = ARG_PREFIX + ['client', '192.168.0.105', PORT, str(request_size), str(request_delay_ms), str(n_requests_per_client)]
            proc = Popen(args, stdout=PIPE, stderr=PIPE, encoding='utf8')
            client_procs.append(proc)
    
        for i, proc in enumerate(client_procs):
            outs, errs = proc.communicate()
            if errs != '':
                print('Error of client', i)
                print(errs)
                continue
            assert proc.returncode == 0
            # Конкатенируем клиентов
            client_out.write(outs)
    
    return experiment_id

In [16]:
by_request_size = []
for k in tqdm([2 ** i for i in range(4, 15)]):
    id = run_experiment(
        request_size=k,
        n_clients=fixed_n_clients,
        request_delay_ms=fixed_request_delay_ms,
        n_requests_per_client=fixed_n_requests_per_client)
    by_request_size.append(id)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:31<00:00,  2.89s/it]


In [17]:
by_n_clients = []
for k in tqdm([1, 2, 5, 10, 20, 50, 100, 200]):
    id = run_experiment(
        request_size=fixed_request_size,
        n_clients=k,
        request_delay_ms=fixed_request_delay_ms,
        n_requests_per_client=fixed_n_requests_per_client)
    by_n_clients.append(id)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:32<00:00,  4.10s/it]


In [18]:
by_request_delay_ms = []
for k in tqdm([0, 10, 20, 50, 100, 200]):
    id = run_experiment(
        request_size=fixed_request_size,
        n_clients=fixed_n_clients,
        request_delay_ms=k,
        n_requests_per_client=fixed_n_requests_per_client)
    by_request_delay_ms.append(id)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:15<00:00,  2.56s/it]


In [19]:
by_n_requests_per_client = []
for k in tqdm([10, 20, 50, 100, 200, 500]):
    id = run_experiment(
        request_size=fixed_request_size,
        n_clients=fixed_n_clients,
        request_delay_ms=fixed_request_delay_ms,
        n_requests_per_client=k)
    by_n_requests_per_client.append(id)

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:23<00:00, 13.87s/it]


In [20]:
experiment_descs.to_csv('results/different_computers/client.csv', index_label='experiment_id')